In [ ]:
#import pytesseract #detection of characters
from ultralytics import YOLO   
import cv2
import matplotlib.pyplot as plt
#import openpyxl
import os
import sqlite3   
from ultralytics.utils.plotting import Annotator
import easyocr
reader=easyocr.Reader(['en'])

In [ ]:
from platform import python_version

print(python_version())

3.12.4


In [ ]:
# Function to create the table if it doesn't exist
def create_table():
    # Connect to the SQLite database
    conn = sqlite3.connect("numberplate.db")
    # Create a cursor object to execute SQL statements
    c = conn.cursor()
    # Create the "vehicles" table with two columns: "vehicle_number" and "bike_image_path"
    # The "IF NOT EXISTS" clause ensures that the table is only created if it doesn't already exist
    c.execute("CREATE TABLE IF NOT EXISTS vehicles (vehicle_number TEXT, bike_image_path TEXT)")
    # Commit the changes to the database
    conn.commit()
    # Close the database connection
    conn.close()


In [ ]:
# Function to insert a record into the "vehicles" table
def insert_record(vehicle_number, bike_image_path):
    # Connect to the SQLite database
    conn = sqlite3.connect("numberplate.db")
    # Create a cursor object to execute SQL statements
    c = conn.cursor()
    # Insert the record into the "vehicles" table using parameterized SQL statement
    c.execute("INSERT INTO vehicles VALUES (?, ?)", (vehicle_number, bike_image_path))
    # Commit the changes to the database
    conn.commit()
    # Close the database connection
    conn.close()

In [ ]:
# workbook = openpyxl.Workbook()
# worksheet = workbook.active

In [ ]:
# Initialize YOLO models
person_bike_model = YOLO(r"C:\Users\User\OneDrive\Desktop\Final project\person_bike_model.pt")
helmet_model = YOLO(r"C:\Users\User\OneDrive\Desktop\Final project\helmet_model.pt")
number_plate_model = YOLO(r"C:\Users\User\OneDrive\Desktop\Final project\number_plate_model.pt")

In [ ]:
# Set up Tesseract OCR
#pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


In [ ]:
%pip install opencv-python==4.8.1.78


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#tesseract ocr alternative
# import easyocr

# reader = easyocr.Reader(['en'])
# extract_info = reader.readtext(image_path)

# for el in extract_info:
#    print(el)

In [ ]:
output_dir = r"C:\Users\User\OneDrive\Desktop\Final project\Output"  # Directory to save the output images for image folder


In [ ]:

# img=r"C:\Users\User\OneDrive\Desktop\Final project\image3.jpg"
# #model test
# person_bike_results = person_bike_model.predict(img, show=True, save=True)
# h_results = helmet_model.predict(img, show=True, save=True)
# n_results = number_plate_model.predict(img, show=True, save=True)


In [8]:

import os

# Path to the folder containing images
input_folder = r"C:\Users\User\OneDrive\Desktop\Final project\otest\otest\otest"


# Get a list of all image files in the input folder
image_files = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith(('.jpg', '.jpeg', '.png'))]

# Process each image in the folder
for image_file in image_files:
    # Load the image
    img = cv2.imread(image_file)

    # Process frame
    #img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect person on a bike
    person_bike_results = person_bike_model.predict(img, show=True, save=True)
    
    # Process each detection result
    for r in person_bike_results:
        boxes = r.boxes
        # Filter detections for person on a bike
        for box in boxes:
            cls = box.cls
            print(person_bike_model.names[int(cls)], person_bike_model.names[int(cls)] == "person_bike")
            if person_bike_model.names[int(cls)] == "person_bike":
                # Crop person on a bike image
                x1, y1, x2, y2 = box.xyxy[0]
                person_bike_image = img[int(y1):int(y2), int(x1):int(x2)]

                # Detect helmet on the person
                helmet_results = helmet_model.predict(person_bike_image, show=True, save=True)

                # Process each helmet detection result
                for hr in helmet_results:
                    h_boxes = hr.boxes
                    # Filter detections for no helmet
                    for h_bo in h_boxes:
                        h_cls = h_bo.cls
                        if not helmet_model.names[int(h_cls)] == "With Helmet" :
                            # Extract number plate from the person bike image
                            number_plate_results = number_plate_model.predict(person_bike_image, show=True, save=True)

                            # Process each number plate detection result
                            for npr in number_plate_results:
                                np_boxes = npr.boxes
                                # Filter detections for number plate
                                for np_box in np_boxes:
                                    np_cls = np_box.cls
                                    print(number_plate_model.names[int(np_cls)])
                                    if number_plate_model.names[int(np_cls)] == "License_Plate":
                                        # Crop number plate image
                                        np_x1, np_y1, np_x2, np_y2 = np_box.xyxy[0]
                                        number_plate_image = person_bike_image[int(np_y1):int(np_y2),
                                                             int(np_x1):int(np_x2)]
                                        # Save the cropped images of persons on bikes with violations
                                        output_file = f"person_bike_violation_{os.path.basename(image_file)}"
                                        output_path = os.path.join(output_dir, output_file)
                                        cv2.imwrite(output_path, person_bike_image)

                                        # Save the cropped number plate image
                                        output_file_number_plate = f"number_plate_violation_{os.path.basename(image_file)}"
                                        output_path_number_plate = os.path.join(output_dir, output_file_number_plate)
                                        cv2.imwrite(output_path_number_plate, number_plate_image)
                                        results=reader.readtext(number_plate_image)
                                        text=" ".join([res[1] for res in results]) if results else "...." 
                                        #gray = cv2.cvtColor(number_plate_image, cv2.COLOR_BGR2GRAY)
                                        #text = EasyOCR.image_to_string(gray)
                                        # Example usage
                                        # Create the "vehicles" table if it doesn't exist
                                        create_table()
                                        # Insert two records into the "vehicles" table
                                        insert_record(text, output_path)
                                        # Print the extracted text
                                        print("Number Plate Text:", text)


0: 416x416 2 person_bikes, 2754.9ms
Speed: 38.7ms preprocess, 2754.9ms inference, 40.6ms postprocess per image at shape (1, 3, 416, 416)
Results saved to runs\detect\predict112
person_bike True

0: 416x160 1 Without Helmet, 205.2ms
Speed: 9.6ms preprocess, 205.2ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 160)
Results saved to runs\detect\predict113

0: 416x160 1 License_Plate, 895.0ms
Speed: 0.0ms preprocess, 895.0ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 160)
Results saved to runs\detect\predict114
License_Plate
Number Plate Text: ....
person_bike True

0: 416x224 2 Without Helmets, 165.2ms
Speed: 8.0ms preprocess, 165.2ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 224)
Results saved to runs\detect\predict113

0: 416x224 (no detections), 1085.1ms
Speed: 38.8ms preprocess, 1085.1ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 224)
Results saved to runs\detect\predict114

0: 416x224 (no detections), 1176.0ms
Speed:

KeyboardInterrupt: 


0: 416x256 1 person_bike, 2092.5ms
Speed: 9.1ms preprocess, 2092.5ms inference, 2.6ms postprocess per image at shape (1, 3, 416, 256)
person_bike True

0: 416x192 1 Without Helmet, 175.1ms
Speed: 3.0ms preprocess, 175.1ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 192)

0: 416x192 (no detections), 1037.7ms
Speed: 2.0ms preprocess, 1037.7ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 192)

0: 416x256 1 person_bike, 1055.3ms
Speed: 3.0ms preprocess, 1055.3ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 256)
person_bike True

0: 416x192 1 Without Helmet, 161.7ms
Speed: 3.0ms preprocess, 161.7ms inference, 2.9ms postprocess per image at shape (1, 3, 416, 192)

0: 416x192 (no detections), 815.9ms
Speed: 3.0ms preprocess, 815.9ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 192)

0: 416x256 1 person_bike, 1118.9ms
Speed: 3.0ms preprocess, 1118.9ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 256)
person_bike True


TypeError: 'NoneType' object is not subscriptable

In [9]:


# Output directory
output_dir1 = r"C:\Users\User\OneDrive\Desktop\Final project\output_dir1"
os.makedirs(output_dir1, exist_ok=True)

# Open video
video_capture = cv2.VideoCapture(r'C:\Users\User\OneDrive\Desktop\Final project\project video.mp4')

while True:
    ret, frame = video_capture.read()
    if not ret or frame is None:
        print("End of video or can't read frame. Exiting...")
        break

    # Detect person on a bike
    person_bike_results = person_bike_model.predict(frame)

    for r in person_bike_results:
        boxes = r.boxes
        for box in boxes:
            cls = box.cls
            if person_bike_model.names[int(cls)] == "person_bike":
                x1, y1, x2, y2 = box.xyxy[0]
                h, w = frame.shape[:2]
                x1 = max(0, int(x1))
                y1 = max(0, int(y1))
                x2 = min(w, int(x2))
                y2 = min(h, int(y2))
                person_bike_image = frame[y1:y2, x1:x2]

                if person_bike_image is None or person_bike_image.size == 0:
                    print("Invalid person-bike crop. Skipping...")
                    continue

                # Detect helmet
                helmet_results = helmet_model.predict(person_bike_image)

                for hr in helmet_results:
                    h_boxes = hr.boxes
                    for h_bo in h_boxes:
                        h_cls = h_bo.cls
                        if helmet_model.names[int(h_cls)] != "With Helmet":
                            # Detect number plate
                            number_plate_results = number_plate_model.predict(person_bike_image)

                            for npr in number_plate_results:
                                np_boxes = npr.boxes
                                for np_box in np_boxes:
                                    np_cls = np_box.cls
                                    if number_plate_model.names[int(np_cls)] == "License_Plate":
                                        np_x1, np_y1, np_x2, np_y2 = np_box.xyxy[0]
                                        h2, w2 = person_bike_image.shape[:2]
                                        np_x1 = max(0, int(np_x1))
                                        np_y1 = max(0, int(np_y1))
                                        np_x2 = min(w2, int(np_x2))
                                        np_y2 = min(h2, int(np_y2))
                                        number_plate_image = person_bike_image[np_y1:np_y2, np_x1:np_x2]

                                        if number_plate_image is None or number_plate_image.size == 0:
                                            print("Invalid number plate crop. Skipping...")
                                            continue

                                        # Save cropped image
                                        output_file = f"violation_{len(os.listdir(output_dir1)) + 1}.jpg"
                                        output_path = os.path.join(output_dir1, output_file)
                                        cv2.imwrite(output_path, person_bike_image)

                                        # OCR
                                        results = reader.readtext(number_plate_image)
                                        text = " ".join([res[1] for res in results]) if results else "N/A"
                                        print("Number Plate Text:", text)

                                        # DB functions (make sure you define these elsewhere)
                                        create_table()
                                        insert_record(text, output_path)

# Cleanup
video_capture.release()
cv2.destroyAllWindows()



0: 416x256 1 person_bike, 1545.0ms
Speed: 0.0ms preprocess, 1545.0ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 256)

0: 416x192 1 Without Helmet, 216.7ms
Speed: 8.6ms preprocess, 216.7ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 192)

0: 416x192 (no detections), 828.5ms
Speed: 0.0ms preprocess, 828.5ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 192)

0: 416x256 1 person_bike, 1122.1ms
Speed: 0.0ms preprocess, 1122.1ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 256)

0: 416x192 1 Without Helmet, 178.5ms
Speed: 8.1ms preprocess, 178.5ms inference, 8.0ms postprocess per image at shape (1, 3, 416, 192)

0: 416x192 (no detections), 787.7ms
Speed: 5.9ms preprocess, 787.7ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 192)

0: 416x256 1 person_bike, 1024.1ms
Speed: 0.0ms preprocess, 1024.1ms inference, 8.0ms postprocess per image at shape (1, 3, 416, 256)

0: 416x192 1 Without Helmet, 142.4ms
Speed: 8.3ms pr

KeyboardInterrupt: 